### MMLU

In [25]:
from datasets import load_dataset

ds = load_dataset("cais/mmlu", "all")
## Stratified sample on 'subject' field
MAX_PER_SUBJ = 50

test_ds = ds['test'].to_pandas()
test_sample = test_ds.groupby('subject', group_keys=False).apply(lambda x: x.sample(min(len(x), MAX_PER_SUBJ), random_state=42)).reset_index(drop=True)


test_sample.to_json('../Self-Distillation/data/mmlu/eval_data.json', orient='records', lines=True)

/tmp/ipykernel_1398915/487230862.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_sample = test_ds.groupby('subject', group_keys=False).apply(lambda x: x.sample(min(len(x), MAX_PER_SUBJ), random_state=42)).reset_index(drop=True)


In [27]:
test_ds.subject

0        abstract_algebra
1        abstract_algebra
2        abstract_algebra
3        abstract_algebra
4        abstract_algebra
               ...       
14037     world_religions
14038     world_religions
14039     world_religions
14040     world_religions
14041     world_religions
Name: subject, Length: 14042, dtype: object

### Tools

In [100]:
import pandas as pd
import os

train_df = pd.read_json('../Self-Distillation/data/tooluse_data/train_data.json')
print("N Rows: ", len(train_df))
print("Columns: ", train_df.columns)

subsample_df = train_df.sample(1000, random_state=42)
os.makedirs("../Self-Distillation/data/tooluse_train/", exist_ok=True)

# subsample_df.to_json('../Self-Distillation/data/tooluse_train/eval_data.json', orient='records')

N Rows:  4046
Columns:  Index(['prompt', 'name', 'description', 'nl_documentation', 'instruction',
       'golden_answer', 'golden_response'],
      dtype='object')


In [101]:
train_df

,prompt,name,description,nl_documentation,instruction,golden_answer,golden_response
0,Your task is to answer the user's question usi...,Httpbin,A Simple HTTP Request & Response Service,sendHttpRequest: Send an HTTP request with the...,"I'm troubleshooting some requests, so can you ...","[{'Action': 'sendHttpRequest', 'Action_Input':...",[ I need to use the sendHttpRequest tool to se...
1,Your task is to answer the user's question usi...,Httpbin,A Simple HTTP Request & Response Service,sendHttpRequest: Send an HTTP request with the...,Could you send a GET request to the following ...,"[{'Action': 'getClientRequestData', 'Action_In...",[ I need to use the getClientRequestData tool ...
2,Your task is to answer the user's question usi...,Httpbin,A Simple HTTP Request & Response Service,sendHttpRequest: Send an HTTP request with the...,I have a feeling that some proxy headers aren'...,"[{'Action': 'sendHttpRequest', 'Action_Input':...",[ I need to send an HTTP request with the spec...
3,Your task is to answer the user's question usi...,Httpbin,A Simple HTTP Request & Response Service,sendHttpRequest: Send an HTTP request with the...,I'd like to see how my application responds to...,"[{'Action': 'simulateStatusCode', 'Action_Inpu...",[ I need to use the simulateStatusCode tool to...
4,Your task is to answer the user's question usi...,Httpbin,A Simple HTTP Request & Response Service,sendHttpRequest: Send an HTTP request with the...,"I need to test how my app handles a ""PATCH"" re...","[{'Action': 'sendHttpRequest', 'Action_Input':...",[ I need to use the sendHttpRequest tool to se...
...,...,...,...,...,...,...,...
4041,Your task is to answer the user's question usi...,Bible-api,Free Bible API with multiple languages,getVerse: Retrieve the text of a specific vers...,"Hey, can you search the Bible for the keyword ...","[{'Action': 'search', 'Action_Input': '{""query...",[ I need to use the search tool to find the ke...
4042,Your task is to answer the user's question usi...,Bible-api,Free Bible API with multiple languages,getVerse: Retrieve the text of a specific vers...,I need to find all the available Bible version...,"[{'Action': 'getVersions', 'Action_Input': '{""...",[ I need to use the getVersions tool to find t...
4043,Your task is to answer the user's question usi...,Bible-api,Free Bible API with multiple languages,getVerse: Retrieve the text of a specific vers...,Can you give me the details of all the Bible v...,"[{'Action': 'getVersions', 'Action_Input': '{""...",[ I need to use the getVersions tool to get th...
4044,Your task is to answer the user's question usi...,Bible-api,Free Bible API with multiple languages,getVerse: Retrieve the text of a specific vers...,I want to find a Bible version in Portuguese. ...,"[{'Action': 'getVersions', 'Action_Input': '{""...",[ I need to use the getVersions tool to find t...


In [88]:
subsample_df = pd.read_json('../Self-Distillation/data/tooluse_train/eval_data.json')
subsample_df.prompt[0]

'Your task is to answer the user\'s question using available tools. \nYou have access to the following tools:\nName: Httpbin\nDescription: A Simple HTTP Request & Response Service\nDocumentation:\nsendHttpRequest: Send an HTTP request with the specified method, headers, and data to the Httpbin API for testing purposes.\nParameters: {"method": "Required. string. One of: [GET, POST, PUT, DELETE, HEAD, PATCH]. The HTTP method to use (GET, POST, PUT, DELETE, HEAD, or PATCH).", "url": "Required. string. The endpoint URL to send the request to.", "headers": "Object.  A key-value pair of headers to include in the request.", "data": "Object.  A key-value pair of data to include in the request body."}\nOutput: Successful response.\n - Format: application/json\n - Structure: Object{response: Object{status_code, headers: Object, body}}\ngetClientRequestData: Retrieve the client\'s request data, including headers, form data, uploaded files, and cookies.\nParameters: {"url": "Required. string. The 

In [40]:
import pandas as pd

eval_df = pd.read_json('../Self-Distillation/data/tooluse_data/eval_data.json')
print("N Rows: ", len(eval_df))
print("Columns: ", eval_df.columns)

N Rows:  68
Columns:  Index(['prompt', 'name', 'description', 'nl_documentation', 'instruction',
       'golden_answer'],
      dtype='object')


In [50]:
print(eval_df.prompt[1])

Your task is to answer the user's question using available tools. 
You have access to the following tools:
Name: Axolotl
Description: Collection of axolotl pictures and facts
Documentation:
getRandomAxolotlImage: Retrieve a random axolotl image with information on the image source.
Parameters: {}
Output: Successful response.
 - Format: application/json
 - Structure: Object{url, source, description}
searchAxolotlImages: Search for axolotl images based on specific criteria such as color, gender, and size.
Parameters: {"color": "string. One of: [wild, leucistic, albino]. The color of the axolotl (e.g., 'wild', 'leucistic', 'albino', etc.).", "gender": "string. One of: [male, female]. The gender of the axolotl ('male', 'female').", "size": "string. One of: [small, medium, large]. The size of the axolotl ('small', 'medium', 'large').", "page": "integer. The page number for pagination purposes."}
Output: Successful response.
 - Format: application/json
 - Structure: Object{results: Array[Obj

In [72]:
import pandas as pd

sim_df = pd.read_json('../Self-Distillation/data/tooluse_data/eval_simulated.json')
real_df = pd.read_json('../Self-Distillation/data/tooluse_data/eval_real.json')

print("Simulated Data:")
print("N Rows: ", len(sim_df))
print("Columns: ", sim_df.columns)

print("Real Data:")
print("N Rows: ", len(real_df))
print("Columns: ", real_df.columns)

Simulated Data:
N Rows:  10
Columns:  Index(['Name', 'Description', 'Link', 'Category', 'Introduction', 'Functions',
       'Documentation', 'NLDocumentation', 'Function_Description',
       'Function_Projection', 'Instructions', 'Golden_Answers'],
      dtype='object')
Real Data:
N Rows:  11
Columns:  Index(['Name', 'Description', 'Category', 'Authentication', 'external_tools',
       'Documentation', 'NLDocumentation', 'Function_Description',
       'Function_Projection', 'Instructions', 'Golden_Answers'],
      dtype='object')


In [79]:
sim_df.Instructions.apply(len).sum(), real_df.Instructions.apply(len).sum()

(np.int64(100), np.int64(114))

In [90]:
eval_df = pd.read_json("../Self-Distillation/data/tooluse_data/eval_data.json")
sim_df = pd.read_json("../Self-Distillation/data/tooluse_sim/eval_data.json")

In [91]:
eval_df.columns, sim_df.columns

(Index(['prompt', 'name', 'description', 'nl_documentation', 'instruction',
        'golden_answer'],
       dtype='object'),
 Index(['prompt', 'name', 'description', 'nl_documentation', 'instruction',
        'golden_answer'],
       dtype='object'))

In [97]:
eval_df.iloc[0].prompt == sim_df.iloc[0].prompt

False

In [ ]:
# string difference


In [99]:
print(sim_df.iloc[0].prompt)

Your task is to answer the user's question using available tools. 
You have access to the following tools:
Name: Axolotl
Description: Collection of axolotl pictures and facts
Documentation:
getRandomAxolotlImage: Retrieve a random axolotl image with information on the image source.
Parameters: {}
Output: Successful response.
 - Format: application/json
 - Structure: Object{url, source, description}
searchAxolotlImages: Search for axolotl images based on specific criteria such as color, gender, and size.
Parameters: {"color": "string. One of: [wild, leucistic, albino]. The color of the axolotl (e.g., 'wild', 'leucistic', 'albino', etc.).", "gender": "string. One of: [male, female]. The gender of the axolotl ('male', 'female').", "size": "string. One of: [small, medium, large]. The size of the axolotl ('small', 'medium', 'large').", "page": "integer. The page number for pagination purposes."}
Output: Successful response.
 - Format: application/json
 - Structure: Object{results: Array[Obj